<h1><b>POS Tagging</b></h1>
The notebook is divided into three section -  <i>Data Preparation</i>, <i>Model Definition</i> and the actual <i>Training and Evaluation</i>.
Following these three sections is one more sectionb - <i>Using Langauge Embeddings</i> - where words are converted to word embeddings and concatenated with language vectors.

In [ ]:
!pip install -q conllu
!pip install -q datasets
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00


<h1>Data Preparation</h1>

<h2>Data Fetching, Cleaning and Storing</h2>
In this section, we fetch the data from the URL: 'https://raw.githubusercontent.com/singhakr/Bhojpuri-Magahi-and-Maithili-Linguistic-Resources/main/bhojpuri/pos-tagged/bhojpuri-pos-tagged-ver-1.3.txt'. The data is cleaned and converted to required format and the output is stored in a file "output.txt". You can download the obtained file in your local storage and use it for further procssing, thus eliminating the need of re-reunning this section again.

In [ ]:
import urllib.request

In [ ]:
dataurl = 'https://raw.githubusercontent.com/singhakr/Bhojpuri-Magahi-and-Maithili-Linguistic-Resources/main/bhojpuri/pos-tagged/bhojpuri-pos-tagged-ver-1.3.txt'
filepath = 'bhojpuri-pos-tagged-ver-1.3.txt'
urllib.request.urlretrieve(dataurl, filepath)

('bhojpuri-pos-tagged-ver-1.3.txt',
 <http.client.HTTPMessage at 0x798ff1800d30>)

In [ ]:
with open(filepath, mode="r", encoding="utf-8") as data:
    annotations = data.read()

In [ ]:
i=0
sents=[]
while(i<len(annotations)):
  sent = ""
  while i<len(annotations):
    if annotations[i]!='\n':
      sent = sent + annotations[i]
      i+=1
    else:
      i+=1
      break
  sents.append(sent)

#############
lines = []
for sent in sents:
  if(len(sent)>0 and sent[0]=='<'):
    continue
  elif "document" in sent:
    continue
  else:
    lines.append(sent)

#############
all = ""
for x in lines:
  all = all + '\n' + x

In [ ]:
if "<fs af=',,,,,,,'>" in all:
    all = all.replace("<fs af=',,,,,,,'>", '')
if "\t))" in all:
  all = all.replace("\t))", "100\t))\tSYM")
elif "))" in all:
  all = all.replace("))", "100\t))\tSYM")
if "NNP:?" in all:
  all = all.replace("NNP:?", "NNP")

In [ ]:
all = all[1:]

In [ ]:
outputfile = open('output.txt', 'w')
outputfile.writelines(all)
outputfile.close()

<h2>Dataset Preparation</h2>

In [ ]:
from conllu import parse
with open("output.txt", mode="r", encoding="utf-8-sig") as data_file:
    data = data_file.read()
sentences = parse(data)

In [ ]:
print(type(sentences))
print(sentences[1])

<class 'conllu.models.SentenceList'>
TokenList<नेवता, देखते, मन, चटक, गइल, ।>


In [ ]:
sents=[]
postags=[]
for sentence in sentences:
  sent = ['hi']
  postag = ['-ID-']
  for token in sentence:
    sent.append(token['form'])
    try:
      postag.append(token['lemma'])
    except:
      postag.append("NA")
  sents.append(sent)
  postags.append(postag)

In [ ]:
print(type(sents))
print(sents[1])

<class 'list'>
['hi', 'नेवता', 'देखते', 'मन', 'चटक', 'गइल', '।']


In [ ]:
print(type(postags))
print(postags[1])

<class 'list'>
['-ID-', 'NN', 'VM', 'NN', 'VM', 'VAUX', 'SYM']


<h3> Get Tag Set</h3>

In [ ]:
tags = []
for postag in postags:
  for tag in postag:
    tags.append(tag)

tags = set(tags)
tag2int = {}
int2tag = {}

for i, tag in enumerate(sorted(tags)):
    tag2int[tag] = i+1
    int2tag[i+1] = tag

tag2int['-PAD-'] = 0
int2tag[0] = '-PAD-'

In [ ]:
tag2int

{'-': 1,
 '-ID-': 2,
 'CC': 3,
 'CCP': 4,
 'CL': 5,
 'DEM': 6,
 'ECH': 7,
 'INJ': 8,
 'INTF': 9,
 'JJ': 10,
 'NA': 11,
 'NEG': 12,
 'NEGP': 13,
 'NN': 14,
 'NNP': 15,
 'NP': 16,
 'NST': 17,
 'PRP': 18,
 'PSP': 19,
 'QC': 20,
 'QF': 21,
 'QO': 22,
 'RB': 23,
 'RDP': 24,
 'RP': 25,
 'SYM': 26,
 'UNK': 27,
 'VAUX': 28,
 'VA�': 29,
 'VGF': 30,
 'VM': 31,
 'WQ': 32,
 '-PAD-': 0}

<h3>Convert to Hugging Face DatasetDict</h3>

In [ ]:
from datasets import Dataset
import pandas as pd

data_list = []
for i in range(0, len(sents)-1):
  temp = {'form':sents[i], 'postag':[tag2int[tag] for tag in postags[i]]}
  data_list.append(temp)
dataset = Dataset.from_pandas(pd.DataFrame(data=data_list))

In [ ]:
from datasets import DatasetDict
train_testvalid = dataset.train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['form', 'postag'],
        num_rows: 3436
    })
    test: Dataset({
        features: ['form', 'postag'],
        num_rows: 430
    })
    valid: Dataset({
        features: ['form', 'postag'],
        num_rows: 429
    })
})

<h2>Tokenization</h2>

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "ai4bharat/IndicBERTv2-SS"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenized_inputs = tokenizer(sents, truncation=True, is_split_into_words=True)

tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            new_labels.append(label)

    return new_labels

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['form'], truncation=True, is_split_into_words=True
    )
    all_labels = examples['postag']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset['train'].column_names,
)

Map:   0%|          | 0/3436 [00:00<?, ? examples/s]

Map:   0%|          | 0/430 [00:00<?, ? examples/s]

Map:   0%|          | 0/429 [00:00<?, ? examples/s]

<h2>Data Collation</h2>


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer
)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["valid"], collate_fn=data_collator, batch_size=8
)

<h1>Model Definition</h1>

<h2>Model</h2>

In [ ]:
from transformers import AutoModelForTokenClassification

model_checkpoint = "ai4bharat/IndicBERTv2-SS"
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=int2tag,
    label2id=tag2int,
)

config.json:   0%|          | 0.00/639 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-SS and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<h2>Optimiser and Scheduler</h2>

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.6 MB/s eta 0:00:00


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

<h2>Preprocessor and Evaluator</h2>

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[int2tag[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [int2tag[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
!pip install -q seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

<h1>Training and Evaluation</h1>

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        try:
          true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
          metric.add_batch(predictions=true_predictions, references=true_labels)
        except:
          print(type(labels_gathered))

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )


  0%|          | 0/1290 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -ID- seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: SYM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: CC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PSP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: User

epoch 0: {'precision': 0.8181499034579603, 'recall': 0.7895985092325936, 'f1': 0.8036206896551724, 'accuracy': 0.8413256339442631}
epoch 1: {'precision': 0.8241179568193786, 'recall': 0.7946851726472579, 'f1': 0.8091339939681171, 'accuracy': 0.8440873713281446}
epoch 2: {'precision': 0.8241179568193786, 'recall': 0.7946851726472579, 'f1': 0.8091339939681171, 'accuracy': 0.8440873713281446}


<h1>Using Language Embeddings</h1>

<h2>For One Word</h2>
We get a word embedding for one word, and concatenate language vector to it.

<h3>Getting Word Embedding</h3>

In [ ]:
import torch
from transformers import AutoModel
from transformers import AutoTokenizer

tokenizer_embed = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-SS")
model_embed = AutoModel.from_pretrained("ai4bharat/IndicBERTv2-SS")

# get the embedding vector for the word "example"
example_token_id = tokenizer_embed.convert_tokens_to_ids(['मन'])[0]
example_embedding = model_embed.embeddings.word_embeddings(torch.tensor([example_token_id]))

Some weights of BertModel were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-SS and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(type(example_embedding))
print(example_embedding.shape)

<class 'torch.Tensor'>
torch.Size([1, 768])


In [ ]:
example_embedding_np = example_embedding.detach().numpy()

In [ ]:
print(type(example_embedding_np))
print(example_embedding_np.shape)

<class 'numpy.ndarray'>
(1, 768)


<h3>Concatenate langvec</h3>

In [ ]:
!pip3 install -q lang2vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 5.6 MB/s eta 0:00:00


In [ ]:
import lang2vec.lang2vec as l2v
features = l2v.get_features("hin", "geo")

In [ ]:
print(type(features['hin']))
print(features['hin'][0:2])
print(len(features['hin']))

<class 'list'>
[0.6082000136375427, 0.690500020980835]
299


In [ ]:
import numpy as np
concatenated_embedding = np.concatenate((example_embedding_np[0], features['hin']))

In [ ]:
concatenated_embedding.shape

(1067,)

In [ ]:
concatenated_embedding_tensor = torch.from_numpy(concatenated_embedding)

<h3>Into one Function</h3>

In [ ]:
def word_to_concatenated_embedding(word, langvec):
  temp = []
  temp.append(word)
  example_token_id = tokenizer_embed.convert_tokens_to_ids(temp)[0]
  example_embedding = model_embed.embeddings.word_embeddings(torch.tensor([example_token_id]))
  example_embedding_np = example_embedding.detach().numpy()
  concatenated_embedding = np.concatenate((example_embedding_np[0], langvec))
  return concatenated_embedding

<h2>For the entire Dataset</h2>

In [ ]:
langvec = features['hin']
sents_embedding = []
for sent in sents:
  sent_embedding = [concatenated_embedding]
  for word in sent:
    embedding = word_to_concatenated_embedding(word, langvec)
    sent_embedding.append(np.array(embedding))
  sents_embedding.append(np.array(sent_embedding))

sents_embedding = np.array(sents_embedding)

<ipython-input-107-f23054d6d521>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sents_embedding = np.array(sents_embedding)


In [ ]:
def sentences_to_embedding(sentences, language_code):
  features = l2v.get_features(language_code, "geo")
  langvec = features[language_code]

  sents_embedding = []
  for sent in sentences:
    sent_embedding = [concatenated_embedding]
    for word in sent:
      embedding = word_to_concatenated_embedding(word, langvec)
      sent_embedding.append(np.array(embedding))
    sents_embedding.append(np.array(sent_embedding))

  sents_embedding = np.array(sents_embedding)

  return sents_embedding

In [ ]:
print(type(sents_embedding))
print(sents_embedding.shape)

<class 'numpy.ndarray'>
(4296,)


In [ ]:
print(type(sents_embedding[0]))
print(sents_embedding[0].shape)

<class 'numpy.ndarray'>
(6, 1067)
